# Branch 1: 시계열 처리 방식 비교

## 🎯 분석 목적
다양한 시계열 처리 방법의 성능 비교를 통한 최적 모델 아키텍처 선정

## 📊 실험 구조

### **1-A: 시계열 무시 (Tabular)**
- **XGBoost**: 그래디언트 부스팅 트리
- **Random Forest**: 랜덤 포레스트
- 각 시점을 독립적인 샘플로 처리
- 시간 순서 정보 무시

### **1-B: 고정 윈도우 (Fixed Window)**
- **LSTM (window=30)**: 과거 30 cycle 사용
- **LSTM (window=50)**: 과거 50 cycle 사용
- 시간 순서 정보 활용
- 고정된 길이의 시퀀스

### **1-C: 가변 윈도우 (Variable Window)**
- **TCN (Temporal Convolutional Network)**: 확장 합성곱
- 다양한 길이의 시퀀스 처리 가능
- 장기 의존성 포착

## 🔬 실험 조건
- **데이터**: 원본 데이터 (전처리 없음)
- **센서**: 전체 21개 센서
- **스케일링**: 모델별 최적 설정
  - Tabular (XGBoost, RandomForest): StandardScaler
  - LSTM/TCN: StandardScaler

## 📈 평가 지표
- RMSE, MAE, 과적합, outlier ratio

In [ ]:
# 라이브러리 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Dropout, Conv1D, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 로드

DATA_PATH = '/content/drive/MyDrive/ML project'

train_original = pd.read_csv(f'{DATA_PATH}/FD004_train_df.csv')
test_original = pd.read_csv(f'{DATA_PATH}/FD004_test_df.csv')

# 센서 컬럼
SENSOR_COLS = [col for col in train_original.columns if col.startswith('s_')]

In [ ]:
# LSTM/TCN용 시퀀스 생성 함수
def create_sequences(data, sensors, window):
    X, y = [], []
    for unit in data['unit'].unique():
        unit_data = data[data['unit'] == unit].sort_values('cycle')
        values = unit_data[sensors].values
        targets = unit_data['RUL'].values

        for i in range(len(values) - window + 1):
            X.append(values[i:i+window])
            y.append(targets[i+window-1])

    return np.array(X), np.array(y)

## 1-A: 시계열 무시 (Tabular)

각 시점을 독립적인 샘플로 처리

In [ ]:
# 1-A: Tabular 접근
results = []

# 데이터 준비 (StandardScaler 적용)
scaler_tabular = StandardScaler()
train_scaled = train_original.copy()
test_scaled = test_original.copy()

scaler_tabular.fit(train_original[SENSOR_COLS])
train_scaled[SENSOR_COLS] = scaler_tabular.transform(train_original[SENSOR_COLS])
test_scaled[SENSOR_COLS] = scaler_tabular.transform(test_original[SENSOR_COLS])

X_train_tab = train_scaled[SENSOR_COLS].values
y_train_tab = train_scaled['RUL'].values
X_test_tab = test_scaled[SENSOR_COLS].values
y_test_tab = test_scaled['RUL'].values

In [ ]:
# 1-A-1: XGBoost
model_xgb = XGBRegressor(
    n_estimators=500,
    max_depth=7,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1
)

model_xgb.fit(X_train_tab, y_train_tab, verbose=False)

y_pred_train_xgb = model_xgb.predict(X_train_tab)
y_pred_test_xgb = model_xgb.predict(X_test_tab)

train_rmse_xgb = np.sqrt(mean_squared_error(y_train_tab, y_pred_train_xgb))
test_rmse_xgb = np.sqrt(mean_squared_error(y_test_tab, y_pred_test_xgb))
test_r2_xgb = r2_score(y_test_tab, y_pred_test_xgb)

results.append({
    'category': '1-A: Tabular',
    'model': 'XGBoost',
    'train_rmse': train_rmse_xgb,
    'test_rmse': test_rmse_xgb,
    'test_r2': test_r2_xgb,
    'overfit': test_rmse_xgb - train_rmse_xgb
})

In [ ]:
# 1-A-2: Random Forest
model_rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

model_rf.fit(X_train_tab, y_train_tab)

y_pred_train_rf = model_rf.predict(X_train_tab)
y_pred_test_rf = model_rf.predict(X_test_tab)

train_rmse_rf = np.sqrt(mean_squared_error(y_train_tab, y_pred_train_rf))
test_rmse_rf = np.sqrt(mean_squared_error(y_test_tab, y_pred_test_rf))
test_r2_rf = r2_score(y_test_tab, y_pred_test_rf)

results.append({
    'category': '1-A: Tabular',
    'model': 'Random Forest',
    'train_rmse': train_rmse_rf,
    'test_rmse': test_rmse_rf,
    'test_r2': test_r2_rf,
    'overfit': test_rmse_rf - train_rmse_rf
})

## 1-B: 고정 윈도우 (Fixed Window)

과거 N개 시점의 데이터를 사용하여 현재 RUL을 예측하는 LSTM 방법

In [ ]:
# 데이터 준비 (StandardScaler)
scaler_lstm = StandardScaler()
train_lstm = train_original.copy()
test_lstm = test_original.copy()

scaler_lstm.fit(train_original[SENSOR_COLS])
train_lstm[SENSOR_COLS] = scaler_lstm.transform(train_original[SENSOR_COLS])
test_lstm[SENSOR_COLS] = scaler_lstm.transform(test_original[SENSOR_COLS])

In [ ]:
# 1-B-1: LSTM (window=30)
X_train_lstm30, y_train_lstm30 = create_sequences(train_lstm, SENSOR_COLS, 30)
X_test_lstm30, y_test_lstm30 = create_sequences(test_lstm, SENSOR_COLS, 30)

model_lstm30 = Sequential([
    Input(shape=(30, len(SENSOR_COLS))),
    LSTM(64, activation='tanh', return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

model_lstm30.compile(optimizer='adam', loss='mse')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history30 = model_lstm30.fit(
    X_train_lstm30, y_train_lstm30,
    epochs=50,
    batch_size=512,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=0
)

y_pred_train_lstm30 = model_lstm30.predict(X_train_lstm30, verbose=0).flatten()
y_pred_test_lstm30 = model_lstm30.predict(X_test_lstm30, verbose=0).flatten()

train_rmse_lstm30 = np.sqrt(mean_squared_error(y_train_lstm30, y_pred_train_lstm30))
test_rmse_lstm30 = np.sqrt(mean_squared_error(y_test_lstm30, y_pred_test_lstm30))
test_r2_lstm30 = r2_score(y_test_lstm30, y_pred_test_lstm30)

results.append({
    'category': '1-B: Fixed Window',
    'model': 'LSTM-30',
    'train_rmse': train_rmse_lstm30,
    'test_rmse': test_rmse_lstm30,
    'test_r2': test_r2_lstm30,
    'overfit': test_rmse_lstm30 - train_rmse_lstm30
})

In [ ]:
# 1-B-2: LSTM (window=50)
X_train_lstm50, y_train_lstm50 = create_sequences(train_lstm, SENSOR_COLS, 50)
X_test_lstm50, y_test_lstm50 = create_sequences(test_lstm, SENSOR_COLS, 50)

model_lstm50 = Sequential([
    Input(shape=(50, len(SENSOR_COLS))),
    LSTM(64, activation='tanh', return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

model_lstm50.compile(optimizer='adam', loss='mse')

history50 = model_lstm50.fit(
    X_train_lstm50, y_train_lstm50,
    epochs=50,
    batch_size=512,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=0
)

y_pred_train_lstm50 = model_lstm50.predict(X_train_lstm50, verbose=0).flatten()
y_pred_test_lstm50 = model_lstm50.predict(X_test_lstm50, verbose=0).flatten()

train_rmse_lstm50 = np.sqrt(mean_squared_error(y_train_lstm50, y_pred_train_lstm50))
test_rmse_lstm50 = np.sqrt(mean_squared_error(y_test_lstm50, y_pred_test_lstm50))
test_r2_lstm50 = r2_score(y_test_lstm50, y_pred_test_lstm50)

results.append({
    'category': '1-B: Fixed Window',
    'model': 'LSTM-50',
    'train_rmse': train_rmse_lstm50,
    'test_rmse': test_rmse_lstm50,
    'test_r2': test_r2_lstm50,
    'overfit': test_rmse_lstm50 - train_rmse_lstm50
})

## 1-C: 가변 윈도우 (Variable Window)

TCN(Temporal Convolutional Network)을 사용한 고급 시계열 처리

In [ ]:
# LSTM과 동일한 데이터 사용 (window=30)
X_train_tcn = X_train_lstm30
y_train_tcn = y_train_lstm30
X_test_tcn = X_test_lstm30
y_test_tcn = y_test_lstm30

# TCN 스타일 모델 (1D CNN)
model_tcn = Sequential([
    Input(shape=(30, len(SENSOR_COLS))),
    Conv1D(64, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    Conv1D(32, kernel_size=3, padding='causal', activation='relu'),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

model_tcn.compile(optimizer='adam', loss='mse')

y_pred_train_tcn = model_tcn.predict(X_train_tcn, verbose=0).flatten()
y_pred_test_tcn = model_tcn.predict(X_test_tcn, verbose=0).flatten()

train_rmse_tcn = np.sqrt(mean_squared_error(y_train_tcn, y_pred_train_tcn))
test_rmse_tcn = np.sqrt(mean_squared_error(y_test_tcn, y_pred_test_tcn))
test_r2_tcn = r2_score(y_test_tcn, y_pred_test_tcn)

results.append({
    'category': '1-C: Variable Window',
    'model': 'TCN',
    'train_rmse': train_rmse_tcn,
    'test_rmse': test_rmse_tcn,
    'test_r2': test_r2_tcn,
    'overfit': test_rmse_tcn - train_rmse_tcn
})

## 📊 결과 분석

In [ ]:
df_results = pd.DataFrame(results)
df_results

,category,model,train_rmse,test_rmse,test_r2,overfit
0,1-A: Tabular,XGBoost,45.922928,72.294111,0.384756,26.371183
1,1-A: Tabular,Random Forest,42.423377,72.169013,0.386883,29.745636
2,1-B: Fixed Window,LSTM-30,83.702887,105.053934,-0.389698,21.351047
3,1-B: Fixed Window,LSTM-50,79.623360,104.232411,-0.425439,24.609051
4,1-C: Variable Window,TCN,145.886461,191.426246,-3.614226,45.539786
